# Análisis de fragmentación de tablas/índices

<span style="font-size: 14px;">Para ver si una BBDD necesita que se apliquen técnicas de defragmentación, esto te ayudaría:</span>

- **Fragmentación de índices**: La fragmentación es un indicador de que los índices pueden necesitar mantenimiento. Puedes usar la función sys.dm\_db\_index\_physical\_stats para obtener información sobre la fragmentación de índices en tu base de datos.
    
- <span style="font-size: 14px;"><b>Estadísticas desactualizadas</b>: Las estadísticas desactualizadas pueden llevar a SQL Server a tomar decisiones de optimización subóptimas. Puedes usar la función sys.dm_db_stats_properties para ver cuándo se actualizaron por última vez las estadísticas.</span>
    
- <span style="font-size: 14px;"><b>Rendimiento de la consulta</b>: Si las consultas se están volviendo más lentas con el tiempo, puede ser un indicador de que los índices necesitan mantenimiento.</span>
    

  

Normalmente las operaciones de defragmentación se agendan con SQL Server Agent. Busca si hay algún job que nombre mantenimiento, maintenance, index\_rebuild, etc. que muestre indicios de que hay algo implementado.

  

Si no localizas nada, probablemente no haya estrategia de mantenimiento implementada.

# Fragmentación de índices

In [ ]:
SELECT 
    dbschemas.[name] as 'Schema',
    dbtables.[name] as 'Table',
    dbindexes.[name] as 'Index',
    indexstats.avg_fragmentation_in_percent,
    indexstats.page_count
FROM 
    sys.dm_db_index_physical_stats (DB_ID(), NULL, NULL, NULL, NULL) AS indexstats
INNER JOIN 
    sys.tables dbtables on dbtables.[object_id] = indexstats.[object_id]
INNER JOIN 
    sys.schemas dbschemas on dbtables.[schema_id] = dbschemas.[schema_id]
INNER JOIN 
    sys.indexes AS dbindexes ON dbindexes.[object_id] = indexstats.[object_id]
    AND indexstats.index_id = dbindexes.index_id
WHERE   
    indexstats.database_id = DB_ID()
ORDER BY 
    indexstats.avg_fragmentation_in_percent desc


# Estadisticas desactualizadas

In [ ]:
SELECT 
    OBJECT_NAME(s.object_id) AS ObjectName,
    i.name AS IndexName,
    s.name AS StatName,
    STATS_DATE(s.object_id, s.stats_id) AS LastUpdated,
    sp.modification_counter AS ModificationCounter
FROM 
    sys.stats AS s 
JOIN 
    sys.indexes AS i ON s.object_id = i.object_id AND s.stats_id = i.index_id
CROSS APPLY 
    sys.dm_db_stats_properties(s.object_id, s.stats_id) AS sp
WHERE 
    sp.modification_counter > 0
ORDER BY 
    sp.modification_counter DESC;
